# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#csv_file = os.path.join(output_data
#weather_df.head()          
path = "../output_data/cities.csv"
cities = pd.read_csv(path)
cities.head()

,City,Cloudiness,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,sonqor,0,51,34.7837,47.6004,35.60,3.06
1,hermanus,42,89,-34.4187,19.2345,55.99,8.75
2,busselton,0,81,-33.6500,115.3333,55.00,1.99
3,monteagudo,40,65,-19.8167,-63.9833,80.60,2.95
4,jumla,0,40,29.2747,82.1838,32.23,4.92


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities[["Lat", "Lng"]]
humidity = cities["Humidity"]


In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity = 125, point_radius = 1)
#heat_layer.max_intensity = 125
#heat_layer.point_radius = 1



# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Use Dataframe to find desired temp, wind and cloudiness
vacation_df= cities.loc[(cities['Max Temperature']>=65)&(cities['Max Temperature']<=85)&(cities['Wind Speed']<=15)&(cities['Cloudiness']<=10)]
vacation_df=vacation_df.dropna()
vacation_df


,City,Cloudiness,Humidity,Lat,Lng,Max Temperature,Wind Speed
26,dakar,0,77,14.6937,-17.4441,66.20,9.22
28,san quintin,0,16,13.3034,-4.8956,71.82,9.40
39,cape town,0,36,-36.4130,-71.9028,77.00,11.50
50,sabla,9,19,12.7525,4.7077,73.56,10.60
75,rocha,0,80,-34.4833,-54.3333,66.69,7.61
...,...,...,...,...,...,...,...
1636,beloha,0,82,-25.1667,45.0500,76.33,10.60
1647,san marcos,0,16,13.3034,-4.8956,71.82,9.40
1650,vera cruz,3,51,-29.4593,-60.2126,82.24,3.44
1655,san lawrenz,0,16,13.3034,-4.8956,71.82,9.40


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = vacation_df
hotel_df["Hotel Names"] = ""
#hotel_names=[]

target_radius = 5000
target_type = "lodging"
gkey = g_key
keyword = "hotel"
params = {"radius": target_radius,
    "type": target_type,
    "key": g_key,
    "keyword" : keyword}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
      
    params["location"] = f"{lat},{lng}"
   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   
    hotel_names = requests.get(base_url, params=params).json()

    #hotel_names = hotel_names.json()
    #print(json.dumps(hotel_names, indent=4, sort_keys=True))
    try:
        hotel_df.loc[index, "Hotel Names"] = hotel_names["results"][0]["name"]
    except (KeyError,IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [8]:
hotel_df 

,City,Cloudiness,Humidity,Lat,Lng,Max Temperature,Wind Speed,Hotel Names
26,dakar,0,77,14.6937,-17.4441,66.20,9.22,
28,san quintin,0,16,13.3034,-4.8956,71.82,9.40,
39,cape town,0,36,-36.4130,-71.9028,77.00,11.50,
50,sabla,9,19,12.7525,4.7077,73.56,10.60,
75,rocha,0,80,-34.4833,-54.3333,66.69,7.61,
...,...,...,...,...,...,...,...,...
1636,beloha,0,82,-25.1667,45.0500,76.33,10.60,
1647,san marcos,0,16,13.3034,-4.8956,71.82,9.40,
1650,vera cruz,3,51,-29.4593,-60.2126,82.24,3.44,
1655,san lawrenz,0,16,13.3034,-4.8956,71.82,9.40,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [10]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)

fig=gmaps.figure(map_type="HYBRID")

markers= gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)
fig.add_layer(heat_layer)

fig


# Display figure


NameError: name 'hotel_info' is not defined